In [ ]:
from huggingface_hub import hf_hub_download, notebook_login
notebook_login()

In [ ]:
import pickle
from google.colab import files

In [ ]:
%%capture
%pip install sae-lens

In [ ]:
from sae_lens import SAE

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn, Tensor

from functools import partial
from jaxtyping import Float, Int
from typing import Optional, Callable, Union, List, Tuple

In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer

In [ ]:
import gc
import pickle
import numpy as np
import matplotlib.pyplot as plt
import json
from fnmatch import fnmatch
from pathlib import Path
from typing import NamedTuple, Optional, Callable, Union, List, Tuple
# from jaxtyping import Float, Int
from collections import Counter

import einops
import torch
from torch import Tensor, nn
from huggingface_hub import snapshot_download
from natsort import natsorted
from safetensors.torch import load_model, save_model

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b")
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
import json
with open('toxicity_dataset.json', 'r') as f:
  dataset = json.load(f)

In [ ]:
model = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

model = model.to('cuda').eval()

def get_llm_actvs_batch(model, inputs, layerID, batch_size=32):
    dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'])
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    accumulated = None
    for input_ids, attn_mask in loader:
        batch = {
            'input_ids': input_ids.to(model.device),
            'attention_mask': attn_mask.to(model.device),
            'output_hidden_states': True
        }
        with torch.no_grad():
            out = model(**batch)
            layer_act = out.hidden_states[layerID]  # (batch, seq_len, hidden)

        accumulated = layer_act if accumulated is None else torch.cat((accumulated, layer_act), dim=0)

        del out, layer_act, batch
        torch.cuda.empty_cache()
        gc.collect()

    b, s, h = accumulated.shape
    return accumulated.reshape(b * s, h).cpu()

def extract_and_save_sae(prompt: str,
                         layer_id: 10,
                         output_path: str = "sae_acts.pt"):
    # --- tokenize (no DataLoader needed for single example)
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,
        truncation=False  # or True + max_length=...
    ).to('cuda')

    # --- forward pass (get hidden states)
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
        # hidden_states[layer_id] is [1, seq_len, hidden_size]
        hidden = outputs.hidden_states[layer_id]   # should be [1, N, 2048]

    # --- load SAE, encode
    sae, cfg, sparsity = SAE.from_pretrained(
        release="gemma-scope-2b-pt-res-canonical",
        sae_id=f"layer_{layer_id}/width_16k/canonical",
    )
    sae = sae.to('cuda').eval()

    with torch.no_grad():
        sae_acts = sae.encode(hidden)              # input dims now match
    return sae_acts.cpu()
    # # --- save & print
    # torch.save(sae_acts.cpu(), output_path)


In [ ]:
gemma2b_acts = []
for i in range(150): #155 is the max with the free colab
  if i % 5 ==0:
    print(f"i: {i}")
  acts = extract_and_save_sae(dataset[i]['text'], 10)
  gemma2b_acts.append(acts)
torch.save({'gemma2b': gemma2b_acts}, 'gemma2b_acts.pt')

In [ ]:
torch.save({'gemma2b': gemma2b_acts}, 'gemma2b_acts.pt')

In [ ]:
print(len(gemma2b_acts))

155


# Testing

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from tqdm import tqdm
import time
from sklearn.decomposition import PCA

class SAEApproximatorPyTorch(nn.Module):
    """
    Linear (or optional 1-layer MLP) approximation from B -> A with elastic-net.
    Automatically filters out low-variance features by rebuilding the first layer.
    """
    def __init__(self, n_in, n_out, hidden_dim=2, device='cuda'):
        super().__init__()
        self.device = device
        self.hidden_dim = hidden_dim
        self.n_out = n_out
        # initial layer sizes; will be rebuilt after filtering
        self.initial_in = n_in
        self._build_model(n_in)
        # normalization stats
        self.register_buffer('B_mean', torch.zeros(n_in))
        self.register_buffer('B_std', torch.ones(n_in))
        self.register_buffer('A_mean', torch.zeros(n_out))
        self.register_buffer('A_std', torch.ones(n_out))
        self.normalize = False

    def _build_model(self, in_features):
        dims = [in_features, 1024, 512, self.n_out]
        layers = []
        for i in range(len(dims)-2):
            layers.append(nn.Linear(dims[i], dims[i+1]))
            layers.append(nn.GELU())
        layers.append(nn.Linear(dims[-2], dims[-1]))
        self.model = nn.Sequential(*layers).to(self.device)


    def forward(self, B):
        if self.normalize:
            B = (B - self.B_mean[:B.size(1)]) / (self.B_std[:B.size(1)] + 1e-8)
            A_pred = self.model(B)
            return A_pred * self.A_std + self.A_mean
        return self.model(B)

    def elastic_net_loss(self, A_pred, A_true, alpha, l1_ratio):
        mse = nn.functional.mse_loss(A_pred, A_true)
        l1 = sum(p.abs().sum() for p in self.model.parameters())
        l2 = sum((p**2).sum() for p in self.model.parameters())
        return mse + alpha * (l1_ratio * l1 + (1 - l1_ratio) * l2)

    def fit(self, A, B, alpha=0.01, l1_ratio=0.5, lr=1e-4,
            epochs=2000, batch_size=32, normalize=True,
            verbose=True, patience=150, var_thresh=1e-6):
        A = torch.as_tensor(A, dtype=torch.float32, device=self.device)
        B = torch.as_tensor(B, dtype=torch.float32, device=self.device)

        # filter low-variance features
        B_var = B.var(dim=0)
        keep_idx = (B_var > var_thresh).nonzero(as_tuple=True)[0]
        B = B[:, keep_idx]
        # rebuild model to match filtered input size
        self._build_model(len(keep_idx))

        if normalize:
            self.normalize = True
            self.B_mean = B.mean(dim=0)
            self.B_std = B.std(dim=0)
            self.A_mean = A.mean(dim=0)
            self.A_std = A.std(dim=0)

        loader = DataLoader(TensorDataset(B, A), batch_size=batch_size, shuffle=True)
        optimizer = optim.Adam(self.parameters(), lr=lr)

        best_loss, counter = float('inf'), 0
        history = []
        start = time.time()
        for epoch in range(epochs):
            epoch_loss = 0
            self.train()
            for bB, bA in loader:
                optimizer.zero_grad()
                pred = self.forward(bB)
                loss = self.elastic_net_loss(pred, bA, alpha, l1_ratio)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
            avg_loss = epoch_loss / len(loader)
            history.append(avg_loss)
            if avg_loss < best_loss:
                best_loss, counter = avg_loss, 0
            else:
                counter += 1
            if verbose and (epoch % 50 == 0 or counter == patience):
                print(f"Epoch {epoch}, Loss {avg_loss:.4e}, kept {len(keep_idx)} features")
            if counter >= patience:
                print(f"Early stopping at epoch {epoch}")
                break
        if verbose:
            print(f"Training done in {time.time()-start:.1f}s, best loss {best_loss:.4e}")
        self.keep_idx = keep_idx
        return history

    def predict(self, B):
        B = torch.as_tensor(B, dtype=torch.float32, device=self.device)
        B = B[:, self.keep_idx]
        self.eval()
        with torch.no_grad():
            out = self.forward(B)
        return out.cpu().numpy()

    def evaluate(self, A_true, B):
        A_true = np.asarray(A_true)
        A_pred = self.predict(B)
        return {'mse': np.mean((A_true - A_pred)**2), 'r2': r2_score(A_true, A_pred)}

# Example usage
if __name__ == '__main__':
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Device: {device}")

    data = torch.load('gemma2b_acts.pt')['gemma2b']
    B_list = [x.squeeze(0).mean(dim=0).cpu().numpy() for x in data]
    B = np.stack(B_list)
    pca = PCA(n_components=60)  # or 512, or tune this
    B = pca.fit_transform(B)
    A = B.copy()
    print(f"Data shape: {B.shape}")

    model = SAEApproximatorPyTorch(n_in=B.shape[1], n_out=A.shape[1], hidden_dim=64, device=device)
    history = model.fit(A, B, alpha=0.001, l1_ratio=0.3, lr=5e-4,
                        epochs=300, batch_size=16, var_thresh=1e-5)

    res = model.evaluate(A, B)
    print(f"Final MSE: {res['mse']:.4e}, R2: {res['r2']:.4f}")

    plt.figure()
    plt.semilogy(history)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Curve')
    plt.grid(True)
    plt.show()